In [ ]:
%pip install selenium parsel

^C
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\anfiniti\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # Import Service untuk mendefinisikan path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd
from IPython.display import clear_output
import pickle

# Buat list untuk menyimpan data
data = []

# Tentukan path ke Chromedriver
chromedriver_path = './chromedriver.exe'  # Ubah sesuai lokasi file Anda

# Buat objek Service dengan path driver
service = Service(chromedriver_path)

# Opsi untuk membuka Chrome dalam bahasa Indonesia dan menonaktifkan pemuatan gambar
options = webdriver.ChromeOptions()
options.add_argument("--lang=id")
# options.add_argument("--blink-settings=imagesEnabled=false")  # Menonaktifkan pemuatan gambar
options.add_argument("--headless")  # Tambahkan jika ingin menjalankan secara headless
options.add_argument("--disable-gpu")  # Untuk meningkatkan kompatibilitas headless
options.add_argument("--blink-settings=imagesEnabled=false")  # Menonaktifkan pemuatan gambar
# Inisialisasi webdriver dengan opsi
driver = webdriver.Chrome(service=service, options=options)

# Buka halaman web
driver.get("https://maps.app.goo.gl/vLdfNhCKiqRakYck8")

time.sleep(5)
# Tunggu elemen tombol "Lihat Semua Ulasan" muncul
try:

    see_all_reviews_button = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '//button[@role="tab" and contains(@aria-label, "Ulasan untuk")]'))
    )
    see_all_reviews_button.click()

except Exception as e:
    print("Tombol 'Lihat Semua Ulasan' tidak ditemukan")
    driver.quit()
    raise
time.sleep(5)
try:
    urutkan = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '//button[contains(@aria-label, "Urutkan ulasan")]'))
    )
    urutkan.click()

except Exception as e:
    print("Tombol 'Urutkan' tidak ditemukan:")
    driver.quit()
    raise
time.sleep(5)
try:
    terbaru = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="fxNQSd" and @data-index="1"]'))
    )
    terbaru.click()

except Exception as e:
    print("Tombol 'Terbaru' tidak ditemukan:")
    driver.quit()
    raise

def scroll_to_load_reviews(max_retries=1000):
    retries = 0
    while retries < max_retries:
        try:
            # Tunggu hingga elemen scrollable muncul
            scrollable_div = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf XiKgde "]'))
            )

            # Ambil jumlah ulasan
            jumlah_ulasan_element = driver.find_element(By.XPATH, '//div[@class="fontBodySmall" and contains(text(), "ulasan")]')
            jumlah_ulasan_text = jumlah_ulasan_element.text.split()[0]
            jumlah_ulasan = int(jumlah_ulasan_text.replace('.', ''))
            print('Jumlah Ulasan:', jumlah_ulasan)

            while True:
                # Scroll ke bawah
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
                jumlah_username = len(driver.find_elements(By.XPATH, './/div[@class="d4r55 "]'))
                clear_output(wait=True)
                print('Jumlah Ulasan:', jumlah_username)

                if jumlah_username == jumlah_ulasan: # ini update jumlahnya sesuai dengan gmaps yaa
                    print("Reached the bottom of the reviews.")
                    break

            break  # Keluar dari loop utama jika berhasil

        except Exception as e:
            print(f"Error during scrolling: {e}")
            print("Retrying in 5 seconds...")
            retries += 1
            time.sleep(5)

    if retries == max_retries:
        print("Max retries reached. Exiting...")
        driver.quit()
        raise Exception("Failed to load reviews after multiple retries")

# Scroll untuk memuat review
scroll_to_load_reviews()

# Ambil review
reviews = driver.find_elements(By.XPATH, '//div[@class="jftiEf fontBodyMedium "]')
stars = driver.find_elements(By.XPATH, '//span[@class="kvMYJc"]')

# Proses data
for review, star in zip(reviews, stars):
    try:
        username = review.find_element(By.XPATH, './/div[@class="d4r55 "]').text
    except:
        username = "Anonymous"
    try:
        review_text = review.find_element(By.XPATH, './/span[@class="wiI7pd"]').text
    except:
        review_text = ""
    try:
        star_count = star.get_attribute("aria-label").split()[0]  # Ambil angka bintang
    except:
        star_count = "0"

    # Tambahkan data ke list
    data.append([username, review_text, star_count])

# Tutup browser
driver.quit()
# Save data to cache
def save_to_cache(data, filename='cache.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

# Load data from cache
def load_from_cache(filename='cache.pkl'):
    try:
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return []

# Save data to cache before quitting
save_to_cache(data)

# Load data from cache if available
data = load_from_cache()

# Continue processing reviews
try:
    # Scroll untuk memuat review
    scroll_to_load_reviews()

    # Ambil review
    reviews = driver.find_elements(By.XPATH, '//div[@class="jftiEf fontBodyMedium "]')
    stars = driver.find_elements(By.XPATH, '//span[@class="kvMYJc"]')

    # Proses data
    for review, star in zip(reviews, stars):
        try:
            username = review.find_element(By.XPATH, './/div[@class="d4r55 "]').text
        except:
            username = "Anonymous"
        try:
            review_text = review.find_element(By.XPATH, './/span[@class="wiI7pd"]').text
        except:
            review_text = ""
        try:
            star_count = star.get_attribute("aria-label").split()[0]  # Ambil angka bintang
        except:
            star_count = "0"

        # Tambahkan data ke list
        data.append([username, review_text, star_count])

    # Save data to cache after processing
    save_to_cache(data)

except Exception as e:
    print(f"Error: {e}")
    save_to_cache(data)
    driver.quit()
    raise

# Tutup browser
driver.quit()

Jumlah Ulasan: 2420
Error during scrolling: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: headless chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A4A813+48355]
	(No symbol) [0x009DC4B1]
	(No symbol) [0x008E5358]
	(No symbol) [0x008CD293]
	(No symbol) [0x0092E37B]
	(No symbol) [0x0093C473]
	(No symbol) [0x0092A536]
	(No symbol) [0x009082DC]
	(No symbol) [0x009093DD]
	GetHandleVerifier [0x00CAAABD+2539405]
	GetHandleVerifier [0x00CEA78F+2800735]
	GetHandleVerifier [0x00CE456C+2775612]
	GetHandleVerifier [0x00AD51E0+616112]
	(No symbol) [0x009E5F8C]
	(No symbol) [0x009E2328]
	(No symbol) [0x009E240B]
	(No symbol) [0x009D4FF7]
	BaseThreadInitThunk [0x76F17BA9+25]
	RtlInitializeExceptionChain [0x77CEC0CB+107]
	RtlClearBits [0x77CEC04F+191]

Retrying in 5 seconds...
Error during scrolling: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: headless

Exception: Failed to load reviews after multiple retries

In [2]:
# Buat DataFrame dari data
df_reviews = pd.DataFrame(data, columns=['Name', 'content', 'score'])
df_reviews.to_csv('place_reviews.csv', index=False)

In [3]:
df_reviews2 = pd.read_csv('Safari Beach Batang reviews.csv')

In [4]:
print(len(df_reviews))
print(len(df_reviews2))

1793
332


In [5]:
df_reviews.head()

,Name,content,score
0,Ruli CH,Terima kasih RSUD Kajen yang telah mengangkat ...,5
1,Ika Mukaromah,Pelayanan kurang ramah,2
2,Aqila Roati,Alhamdulillah dn terimakasih IGD walupun ruang...,5
3,Samx Qoutes,Pelayanan untuk pasien BPJS tolong diperbaiki ...,1
4,kknnewung Asik,RSUD di sini ko dari dulu sampai sekarang ga a...,1
